# Sudoku Game with Pygame 🎮

This notebook implements a Sudoku game using Python and Pygame. It includes:
- A Sudoku puzzle generator
- A backtracking solver
- A graphical interface built with Pygame

Run each cell below to build and launch the game.


In [ ]:
!pip install pygame

import pygame
import sys
from typing import List, Tuple, Optional
import random


## Initialize Pygame

Pygame must be initialized before using its modules.


In [ ]:
pygame.init()


## SudokuBoard Class

This class manages the logic of the Sudoku puzzle:
- Board generation
- Valid move checking
- Solving with backtracking
- Puzzle creation by removing numbers


In [ ]:
class SudokuBoard:
    def __init__(self):
        self.board = [[0 for _ in range(9)] for _ in range(9)]
        self.fixed_cells = set()
        self.generate_puzzle()
        

    
    def is_valid_placement(self, row: int, col: int, num: int) -> bool:
        if num in self.board[row]:
            return False
        if num in [self.board[i][col] for i in range(9)]:
            return False
        box_row, box_col = 3 * (row // 3), 3 * (col // 3)
        for i in range(box_row, box_row + 3):
            for j in range(box_col, box_col + 3):
                if self.board[i][j] == num:
                    return False
        return True
    
    def find_empty_cell(self) -> Optional[Tuple[int, int]]:
        for i in range(9):
            for j in range(9):
                if self.board[i][j] == 0:
                    return (i, j)
        return None
    
    def solve(self) -> bool:
        empty = self.find_empty_cell()
        if not empty:
            return True
        row, col = empty
        for num in range(1, 10):
            if self.is_valid_placement(row, col, num):
                self.board[row][col] = num
                if self.solve():
                    return True
                self.board[row][col] = 0
        return False
        
    def is_board_complete_and_valid(self) -> bool:
        for i in range(9):
            for j in range(9):
                num = self.board[i][j]
                if num == 0:
                    return False
                self.board[i][j] = 0  # Temporarily clear
                if not self.is_valid_placement(i, j, num):
                    self.board[i][j] = num  # Restore before returning
                    return False
                self.board[i][j] = num
        return True

    
    def generate_puzzle(self, difficulty: int = 5) -> None:
        self.board = [[0 for _ in range(9)] for _ in range(9)]
        self.fixed_cells.clear()
        for i in range(0, 9, 3):
            nums = list(range(1, 10))
            random.shuffle(nums)
            for row in range(3):
                for col in range(3):
                    self.board[i + row][i + col] = nums[row * 3 + col]
                    self.fixed_cells.add((i + row, i + col))
        self.solve()
        cells = [(i, j) for i in range(9) for j in range(9)]
        random.shuffle(cells)
        for i, j in cells[:difficulty]:
            if (i, j) not in self.fixed_cells:
                self.board[i][j] = 0
    
    def is_cell_fixed(self, row: int, col: int) -> bool:
        return (row, col) in self.fixed_cells


## SudokuGUI Class

This class builds the graphical interface using Pygame.
- Draws the board
- Handles mouse clicks and key presses
- Allows the user to play the game


In [6]:
class SudokuGUI:
    def __init__(self):
        self.WINDOW_SIZE = 540
        self.GRID_SIZE = 9
        self.CELL_SIZE = self.WINDOW_SIZE // self.GRID_SIZE
        self.game_won = False
        self.WHITE = (255, 255, 255)
        self.BLACK = (0, 0, 0)
        self.GRAY = (200, 200, 200)
        self.BLUE = (0, 0, 255)
        self.RED = (255, 0, 0)
        
        self.screen = pygame.display.set_mode((self.WINDOW_SIZE, self.WINDOW_SIZE))
        pygame.display.set_caption(" بازی سودوکو")
        self.font = pygame.font.Font(None, 40)
        
        self.selected_cell = None
        self.board = SudokuBoard()
    
    def draw_grid(self):
        for i in range(self.GRID_SIZE):
            for j in range(self.GRID_SIZE):
                x = j * self.CELL_SIZE
                y = i * self.CELL_SIZE
                color = self.WHITE if (i, j) != self.selected_cell else self.GRAY
                pygame.draw.rect(self.screen, color, (x, y, self.CELL_SIZE, self.CELL_SIZE))
                pygame.draw.rect(self.screen, self.BLACK, (x, y, self.CELL_SIZE, self.CELL_SIZE), 1)
                if self.board.board[i][j] != 0:
                    num = str(self.board.board[i][j])
                    color = self.BLACK if self.board.is_cell_fixed(i, j) else self.BLUE
                    text = self.font.render(num, True, color)
                    text_rect = text.get_rect(center=(x + self.CELL_SIZE//2, y + self.CELL_SIZE//2))
                    self.screen.blit(text, text_rect)
        for i in range(0, self.WINDOW_SIZE, self.CELL_SIZE * 3):
            pygame.draw.line(self.screen, self.BLACK, (i, 0), (i, self.WINDOW_SIZE), 3)
            pygame.draw.line(self.screen, self.BLACK, (0, i), (self.WINDOW_SIZE, i), 3)
    
    def handle_click(self, pos: Tuple[int, int]) -> None:
        x, y = pos
        row = y // self.CELL_SIZE
        col = x // self.CELL_SIZE
        if 0 <= row < 9 and 0 <= col < 9:
            if not self.board.is_cell_fixed(row, col):
                self.selected_cell = (row, col)
    
    def handle_key(self, key: int) -> None:
        if self.selected_cell is None:
            return
        row, col = self.selected_cell
        if key == pygame.K_BACKSPACE or key == pygame.K_DELETE:
            self.board.board[row][col] = 0
        elif pygame.K_1 <= key <= pygame.K_9:
            num = key - pygame.K_0
            if self.board.is_valid_placement(row, col, num):
                self.board.board[row][col] = num


    def display_win_message(self):
        win_text = self.font.render("You Win!", True, self.RED)
        rect = win_text.get_rect(center=(self.WINDOW_SIZE // 2, self.WINDOW_SIZE // 2))
        self.screen.blit(win_text, rect)


        
    def run(self):
        running = True
        while running:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False
                elif not self.game_won:
                    if event.type == pygame.MOUSEBUTTONDOWN:
                        self.handle_click(event.pos)
                    elif event.type == pygame.KEYDOWN:
                        self.handle_key(event.key)
            
            self.screen.fill(self.WHITE)
            self.draw_grid()

            # Check win condition and set flag
            if not self.game_won and self.board.is_board_complete_and_valid():
                self.game_won = True

            # Always show win message after winning
            if self.game_won:
                self.display_win_message()

            pygame.display.flip()


## Launch the Game

Run the following cell to launch the Sudoku game in a Pygame window.


In [7]:
if __name__ == "__main__":
    game = SudokuGUI()
    game.run()
